<a href="https://colab.research.google.com/github/foundationequities/pytorch-forecasting/blob/master/Stock_Earnings_GooglePytrends_8_27_22_LOCK_May26th'23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!pip install pytorch-forecasting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pytorch_forecasting

In [4]:
import copy
from pathlib import Path

import numpy as np
import pandas as pd
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
import torch

import pytorch_forecasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, CrossEntropy, MAE, MultiLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [5]:
#%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_Jan22/Classifier
%cd /content/gdrive/My\ Drive/Forecaster/GooglePytrends
!ls

/content/gdrive/My Drive/Forecaster/GooglePytrends
 BiotechEarnings_Clean_Pytrends_8-14-22.csv		    NoNews
 Biotech_Earnings+GuidanceFlag+Pytrends_clean_8-26-22.csv   OIH_15T.csv.gz
 Biotech_Earnings_TestDummyChangeValue_v1.csv		    OIH_1D.csv.gz
'Copy of Earnings_TEST_AllData_v1.csv'			    OIH_1H.csv.gz
 Earnings_TEST_AllData_v12-20-22.csv			    OIH_1T.csv.gz
 Earnings_TEST_AllData_v1.csv				    OIH_5T.csv.gz
 GooglePytrends_10-21-22.csv				    OIH_adjusted.txt
 GooglePytrends_Clean_8-9-22.csv			    Pytrends_Index
 GooglePyTrends.csv					    wandb
 lightning_logs


In [6]:
#df_train = pd.concat(map(pd.read_csv, ['2018_BTCUSDT.csv', '2019_BTCUSDT.csv','2020_BTCUSDT.csv']))
#df_train = pd.read_csv('biotech_earnings_v12-14-21_NoPDLI.csv',  encoding='latin1')

df_train = pd.read_csv('Biotech_Earnings+GuidanceFlag+Pytrends_clean_8-26-22.csv',  encoding='latin1')

In [7]:
#df_train=df_train2
df_train.describe()

,Unnamed: 0,Open5min,High5min,Low5min,Close5min,Open5minPercent,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,...,PrevChangePercent,Volatility,MarketCap,SMA_Volume,EstimatedEPS,EPS,Guidance,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent
count,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,...,53030.000000,53030.000000,53030.000000,5.303000e+04,53030.000000,53030.000000,13100.0,53030.000000,53030.000000,53030.000000
mean,26896.386970,35.733666,35.965928,35.491420,35.727320,-0.111771,0.673326,-0.939763,-0.148588,531.063888,...,0.089576,58.903485,7414.315482,9.415021e+05,-0.018345,-0.006621,1.0,339.123176,97.525627,-0.005918
std,15514.093257,61.841386,62.147944,61.515431,61.827762,16.321274,16.775307,16.124076,16.519349,1453.301088,...,3.946340,39.158728,28916.643545,2.084027e+06,0.923097,1.150119,0.0,340.604333,191.021727,0.744194
min,0.000000,0.110300,0.111000,0.110200,0.110200,-61.870000,-60.370000,-63.550000,-61.870000,0.000000,...,-43.197300,0.000000,0.000000,0.000000e+00,-2.980000,-30.600000,1.0,0.000000,0.000000,-3.437798
25%,13473.250000,4.350000,4.380000,4.310000,4.350000,-3.490000,-2.870000,-4.310000,-3.650000,127.300000,...,-1.730400,36.290000,154.000000,1.352750e+05,-0.460000,-0.450000,1.0,83.000000,0.000000,-0.356554
50%,26890.500000,13.120000,13.240000,13.000000,13.110000,-0.180000,0.180000,-0.780000,-0.240000,271.700000,...,0.000000,51.585900,537.000000,3.633420e+05,-0.170000,-0.150000,1.0,221.000000,18.000000,-0.006620
75%,40337.750000,40.500000,40.800000,40.180000,40.500000,2.897500,3.620000,2.240000,2.960000,533.100000,...,1.700800,70.099600,2276.000000,9.486940e+05,0.150000,0.200000,1.0,495.000000,99.000000,0.320124
max,53745.000000,964.930000,967.970000,951.500000,955.475000,1121.580000,1123.930000,1121.580000,1123.930000,78751.700000,...,42.857100,510.079400,381029.000000,5.207490e+07,7.530000,9.150000,1.0,1570.000000,1569.000000,6.262782


In [8]:
df_train2 = df_train.drop(columns=['Open5minPercent', 'MarketCap', 'Open5min', 'High5min', 'Low5min', 'Close5min','Close', 'SMA_Volume', 'Unnamed: 0', 'Guidance'])
#df_train2 = df_train.drop(columns=['Change % (1)', 'Unnamed: 0'])


In [ ]:
#df_train2['Guidance'] = df_train2['Guidance'].fillna(0)

In [9]:
#df_train2 = df_train
df_train2.describe()

,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent
count,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000,53030.000000
mean,0.673326,-0.939763,-0.148588,531.063888,-0.198607,40.040139,0.089576,58.903485,-0.018345,-0.006621,339.123176,97.525627,-0.005918
std,16.775307,16.124076,16.519349,1453.301088,9.294765,131.979757,3.946340,39.158728,0.923097,1.150119,340.604333,191.021727,0.744194
min,-60.370000,-63.550000,-61.870000,0.000000,-59.813100,0.256400,-43.197300,0.000000,-2.980000,-30.600000,0.000000,0.000000,-3.437798
25%,-2.870000,-4.310000,-3.650000,127.300000,-4.498300,5.399400,-1.730400,36.290000,-0.460000,-0.450000,83.000000,0.000000,-0.356554
50%,0.180000,-0.780000,-0.240000,271.700000,-0.257500,14.700000,0.000000,51.585900,-0.170000,-0.150000,221.000000,18.000000,-0.006620
75%,3.620000,2.240000,2.960000,533.100000,4.149500,42.910000,1.700800,70.099600,0.150000,0.200000,495.000000,99.000000,0.320124
max,1123.930000,1121.580000,1123.930000,78751.700000,83.381900,6368.000000,42.857100,510.079400,7.530000,9.150000,1570.000000,1569.000000,6.262782


In [ ]:
df_train2['Pytrends_Sum'] = df_train2['SymbolName_Sum']+df_train2['CompanyName_Sum']
df_train2.head()

,Symbol,Date,Time,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,Pytrends_Sum
0,WBA,2016-01-07,09:30,3.48,0.53,0.82,165.9,1.9339,79.63,-1.5333,24.8742,0.96,1.03,372.0,49.0,-2.461853,421.0
1,WBA,2016-01-07,09:35,1.14,-0.15,-0.10,220.9,1.9339,79.63,-1.5333,24.8742,0.96,1.03,372.0,49.0,-2.461853,421.0
2,WBA,2016-01-07,09:40,0.25,-0.58,-0.11,250.1,1.9339,79.63,-1.5333,24.8742,0.96,1.03,372.0,49.0,-2.461853,421.0
3,WBA,2016-01-07,09:45,0.57,-0.75,-0.06,259.1,1.9339,79.63,-1.5333,24.8742,0.96,1.03,372.0,49.0,-2.461853,421.0
4,WBA,2016-01-07,09:50,0.51,-0.35,0.25,270.6,1.9339,79.63,-1.5333,24.8742,0.96,1.03,372.0,49.0,-2.461853,421.0


In [ ]:
df = df_train2
#Getting all unique symbols - Generating a time series VELOCITY-ACCELERATION like data (per Google Pressure Kaggle test)
symbols = df['Symbol'].unique()
for symbol in symbols:
    #Getting all rows for particular symbol
    symbol_data = df[df['Symbol'] == symbol]

    #Getting all the dates for which the data is present for that symbol
    unique_dates = symbol_data['Date'].unique()
    for date in unique_dates:
        #Getting all the rows for particular date
        data_for_date = symbol_data[symbol_data['Date'] == date]

        for i,r in data_for_date.iterrows():
            prev_close = r['PreviousClosePrice']
            cp5 = r['Close5minPercent']
            new_cp = prev_close + ((prev_close*cp5)/100)
            df.at[i,'Current_price'] = new_cp

            if r['Time'] == '09:30':
                df.at[i,'Velocity'] = 0
                df.at[i,'Acceleration'] = 0
            else:
                vel = df.iloc[i]['Current_price'] - df.iloc[i-1]['Current_price']
                df.at[i,'Velocity'] = vel

                acc = df.iloc[i]['Velocity'] - df.iloc[i-1]['Velocity']
                df.at[i,'Acceleration'] = acc

In [ ]:
#remove values
index_drop = df_train2[(df_train2['PreviousClosePrice'] >= 250)|(df_train2['PreviousClosePrice'] <= 2.00)].index
df_train2.drop(index_drop, inplace=True)
index_drop2 = df_train2[(df_train2['Volatility'] >= 250)|(df_train2['Volatility'] <= 10)].index
df_train2.drop(index_drop2, inplace=True)
#index_drop3 = df_train2[(df_train2['MarketCap'] >= 50000)|(df_train2['MarketCap'] <= 20)].index
#df_train2.drop(index_drop3, inplace=True)

index_drop4 = df_train2[(df_train2['ChangePercent'] >= 25)|(df_train2['ChangePercent'] <= -25)].index
df_train2.drop(index_drop4,inplace=True)
index_drop5 = df_train2[(df_train2['PrevChangePercent'] >= 35)|(df_train2['PrevChangePercent'] <= -35)].index
df_train2.drop(index_drop5,inplace=True)

#clip-renumber max values
df_train2['ChangePercent'] = df_train2['ChangePercent'].clip(-25,25)
#df_train2['EstimatedEPS'] = df_train2['EstimatedEPS'].clip(-8,8)
df_train2['EPS'] = df_train2['EPS'].clip(-8,8)
df_train2['Vol5minPercent'] = df_train2['Vol5minPercent'].clip(0,2000)
df_train = df_train2
df_train.describe()


,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,Current_price,Velocity,Acceleration
count,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000
mean,0.731026,-0.831382,-0.063211,420.997419,-0.101174,34.934393,0.046555,54.183397,-0.044094,-0.025872,337.163840,99.632845,-0.014417,34.932981,-0.002164,-0.000761
std,17.068667,16.406880,16.803191,438.925977,7.525187,43.720303,3.520261,28.465617,0.852029,0.961218,345.438217,194.710250,0.716226,43.940455,0.525724,0.750239
min,-35.510000,-37.280000,-36.570000,0.000000,-24.824400,2.006500,-28.671300,10.196500,-2.980000,-8.000000,0.000000,0.000000,-3.437798,1.549944,-12.312480,-16.005000
25%,-2.780000,-4.210000,-3.540000,135.100000,-4.248400,6.800000,-1.684200,35.300800,-0.480000,-0.470000,79.000000,0.000000,-0.356554,6.749706,-0.060255,-0.100361
50%,0.250000,-0.760000,-0.210000,275.900000,-0.074250,16.600000,0.000000,49.982650,-0.170000,-0.160000,214.000000,20.000000,-0.006620,16.499862,0.000000,0.000000
75%,3.610000,2.220000,2.950000,522.525000,4.152800,45.010000,1.640700,66.328200,0.200000,0.260000,487.000000,100.000000,0.310544,45.103580,0.060322,0.100396
max,1123.930000,1121.580000,1123.930000,2000.000000,24.858800,248.580000,28.191500,248.521500,7.530000,8.000000,1570.000000,1569.000000,6.262782,267.101030,12.056148,18.056688


In [ ]:
df_train.replace([np.inf, -np.inf], np.nan)
df_train.dropna(inplace=True)

In [ ]:
df_train['Date'] = pd.to_datetime(df_train['Date'], errors = 'coerce')
df_train['Time'] = pd.to_datetime(df_train['Time'])


#df_train['Time'] = pd.to_timedelta(df_train['Time'], errors = 'coerce')
#df_train['Time'] = pd.to_timedelta(df_train['Time']+':00').dt.time
df_train['time_idx'] = ((df_train['Time'].dt.hour*60 + df_train['Time'].dt.minute)/5)-114
df_train['time_idx'] = df_train['time_idx'].astype(int)

#df_train = df_train.drop(columns=['Date', 'Time'])

In [ ]:
df_train['year'] = df_train['Date'].dt.year
df_train['month'] = df_train['Date'].dt.month
df_train['day']= df_train['Date'].dt.day
df_train['hour']= df_train['Time'].dt.hour
df_train[['year','month', 'day', 'time_idx']] = df_train[['year','month', 'day', 'time_idx']].astype(int)
df_train[['year','month', 'day','hour']] = df_train[['year','month', 'day', 'hour']].astype(str)

#df_train["EstimatedRevenue"] = df_train["EstimatedRevenue"].astype(float) #can't convert string to float
#df_train["OpenNasdaqChange"] = pd.to_numeric(df_train["OpenNasdaqChange"])
#df_train["OpenNasdaqChange"] = df_train["OpenNasdaqChange"].astype(float) #can't convert string to float
#df_train = df_train.drop(columns = ['Date','Time'])
df_train.describe()

,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,Current_price,Velocity,Acceleration,time_idx
count,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000,46460.000000
mean,0.731026,-0.831382,-0.063211,420.997419,-0.101174,34.934393,0.046555,54.183397,-0.044094,-0.025872,337.163840,99.632845,-0.014417,34.932981,-0.002164,-0.000761,4.500000
std,17.068667,16.406880,16.803191,438.925977,7.525187,43.720303,3.520261,28.465617,0.852029,0.961218,345.438217,194.710250,0.716226,43.940455,0.525724,0.750239,2.872312
min,-35.510000,-37.280000,-36.570000,0.000000,-24.824400,2.006500,-28.671300,10.196500,-2.980000,-8.000000,0.000000,0.000000,-3.437798,1.549944,-12.312480,-16.005000,0.000000
25%,-2.780000,-4.210000,-3.540000,135.100000,-4.248400,6.800000,-1.684200,35.300800,-0.480000,-0.470000,79.000000,0.000000,-0.356554,6.749706,-0.060255,-0.100361,2.000000
50%,0.250000,-0.760000,-0.210000,275.900000,-0.074250,16.600000,0.000000,49.982650,-0.170000,-0.160000,214.000000,20.000000,-0.006620,16.499862,0.000000,0.000000,4.500000
75%,3.610000,2.220000,2.950000,522.525000,4.152800,45.010000,1.640700,66.328200,0.200000,0.260000,487.000000,100.000000,0.310544,45.103580,0.060322,0.100396,7.000000
max,1123.930000,1121.580000,1123.930000,2000.000000,24.858800,248.580000,28.191500,248.521500,7.530000,8.000000,1570.000000,1569.000000,6.262782,267.101030,12.056148,18.056688,9.000000


In [ ]:
#Classifier Targets (0.5% increments up to 25% and everything below 0%)

conditionT = [
  (df_train['ClosePercent'] <= 0),
  (df_train['ClosePercent'] > 0) & (df_train['ClosePercent'] <= 0.5),
  (df_train['ClosePercent'] > 0.5) & (df_train['ClosePercent'] <= 1),
  (df_train['ClosePercent'] > 1) & (df_train['ClosePercent'] <= 1.5),
  (df_train['ClosePercent'] > 1.5) & (df_train['ClosePercent'] <= 2),
  (df_train['ClosePercent'] > 2) & (df_train['ClosePercent'] <= 2.5),
  (df_train['ClosePercent'] > 2.5) & (df_train['ClosePercent'] <= 3),
  (df_train['ClosePercent'] > 3) & (df_train['ClosePercent'] <= 3.5),
  (df_train['ClosePercent'] > 3.5) & (df_train['ClosePercent'] <= 4),
  (df_train['ClosePercent'] > 4) & (df_train['ClosePercent'] <= 4.5),
  (df_train['ClosePercent'] > 4.5) & (df_train['ClosePercent'] <= 5),
  (df_train['ClosePercent'] > 5) & (df_train['ClosePercent'] <= 5.5),
  (df_train['ClosePercent'] > 5.5) & (df_train['ClosePercent'] <= 6),
  (df_train['ClosePercent'] > 6) & (df_train['ClosePercent'] <= 6.5),
  (df_train['ClosePercent'] > 6.5) & (df_train['ClosePercent'] <= 7),
  (df_train['ClosePercent'] > 7) & (df_train['ClosePercent'] <= 7.5),
  (df_train['ClosePercent'] > 7.5) & (df_train['ClosePercent'] <= 8),
  (df_train['ClosePercent'] > 8) & (df_train['ClosePercent'] <= 8.5),
  (df_train['ClosePercent'] > 8.5) & (df_train['ClosePercent'] <= 9),
  (df_train['ClosePercent'] > 9) & (df_train['ClosePercent'] <= 9.5),
  (df_train['ClosePercent'] > 9.5) & (df_train['ClosePercent'] <= 10),
  (df_train['ClosePercent'] > 10) & (df_train['ClosePercent'] <= 10.5),
  (df_train['ClosePercent'] > 10.5) & (df_train['ClosePercent'] <= 11),
  (df_train['ClosePercent'] > 11) & (df_train['ClosePercent'] <= 11.5),
  (df_train['ClosePercent'] > 11.5) & (df_train['ClosePercent'] <= 12),
  (df_train['ClosePercent'] > 12) & (df_train['ClosePercent'] <= 12.5),
  (df_train['ClosePercent'] > 12.5) & (df_train['ClosePercent'] <= 13),
  (df_train['ClosePercent'] > 13) & (df_train['ClosePercent'] <= 13.5),
  (df_train['ClosePercent'] > 13.5) & (df_train['ClosePercent'] <= 14),
  (df_train['ClosePercent'] > 14) & (df_train['ClosePercent'] <= 14.5),
  (df_train['ClosePercent'] > 14.5) & (df_train['ClosePercent'] <= 15),
  (df_train['ClosePercent'] > 15) & (df_train['ClosePercent'] <= 15.5),
  (df_train['ClosePercent'] > 15.5) & (df_train['ClosePercent'] <= 16),
  (df_train['ClosePercent'] > 16) & (df_train['ClosePercent'] <= 16.5),
  (df_train['ClosePercent'] > 16.5) & (df_train['ClosePercent'] <= 17),
  (df_train['ClosePercent'] > 17) & (df_train['ClosePercent'] <= 17.5),
  (df_train['ClosePercent'] > 17.5) & (df_train['ClosePercent'] <= 18),
  (df_train['ClosePercent'] > 18) & (df_train['ClosePercent'] <= 18.5),
  (df_train['ClosePercent'] > 18.5) & (df_train['ClosePercent'] <= 19),
  (df_train['ClosePercent'] > 19) & (df_train['ClosePercent'] <= 19.5),
  (df_train['ClosePercent'] > 19.5) & (df_train['ClosePercent'] <= 20),
  (df_train['ClosePercent'] > 20) & (df_train['ClosePercent'] <= 20.5),
  (df_train['ClosePercent'] > 20.5) & (df_train['ClosePercent'] <= 21),
  (df_train['ClosePercent'] > 21) & (df_train['ClosePercent'] <= 21.5),
  (df_train['ClosePercent'] > 21.5) & (df_train['ClosePercent'] <= 22),
  (df_train['ClosePercent'] > 22) & (df_train['ClosePercent'] <= 22.5),
  (df_train['ClosePercent'] > 22.5) & (df_train['ClosePercent'] <= 23),
  (df_train['ClosePercent'] > 23) & (df_train['ClosePercent'] <= 23.5),
  (df_train['ClosePercent'] > 23.5) & (df_train['ClosePercent'] <= 24),
  (df_train['ClosePercent'] > 24) & (df_train['ClosePercent'] <= 24.5),
  (df_train['ClosePercent'] > 24.5) & (df_train['ClosePercent'] <= 25)]
valueW = [0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5, 9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15, 15.5, 16, 16.5, 17, 17.5, 18, 18.5, 19, 19.5, 20, 20.5, 21, 21.5, 22, 22.5, 23, 23.5, 24, 24.5, 25]
df_train['Target'] = np.select(conditionT, valueW)

df_train.describe()

total = df_train['Target'].sum()
print(total)

In [ ]:
new_column_name = 'FinalTime'
df_train[new_column_name] = ''

#Getting all unique symbols
symbols = df_train['Symbol'].unique()
for symbol in symbols:
    #Getting all rows for particular symbol
    symbol_data = df_train[df_train['Symbol'] == symbol]
    print(f"Running for symbol ;{symbol}")

    #Getting all the dates for which the data is present for that symbol
    unique_dates = symbol_data['Date'].unique()
    for date in unique_dates:
        #Getting all the rows for particular date
        data_for_date = symbol_data[symbol_data['Date'] == date]
        print(f"Running for date: {date}")

#         print(type(data_for_date))
        value = data_for_date.iloc[-1]['Close5minPercent']
        print(f"Adding value: {value}")
        for i,r in data_for_date.iterrows():
            df_train.at[i,new_column_name] = value

#         break

#     break


Streaming output truncated to the last 5000 lines.
Running for date: 2016-03-09T00:00:00.000000000
Adding value: -0.27
Running for date: 2016-11-04T00:00:00.000000000
Adding value: 1.95
Running for date: 2017-03-09T00:00:00.000000000
Adding value: -1.8
Running for date: 2017-05-10T00:00:00.000000000
Adding value: 1.2
Running for date: 2017-11-08T00:00:00.000000000
Adding value: -9.52
Running for date: 2018-03-08T00:00:00.000000000
Adding value: 2.33
Running for date: 2018-05-09T00:00:00.000000000
Adding value: -0.83
Running for date: 2018-08-08T00:00:00.000000000
Adding value: 8.54
Running for date: 2018-11-07T00:00:00.000000000
Adding value: 1.2
Running for date: 2019-03-07T00:00:00.000000000
Adding value: 1.94
Running for date: 2019-05-08T00:00:00.000000000
Adding value: -1.79
Running for symbol ;BCLI
Running for date: 2016-03-10T00:00:00.000000000
Adding value: 1.65
Running for date: 2016-05-10T00:00:00.000000000
Adding value: -0.47
Running for date: 2016-08-11T00:00:00.000000000
Ad

In [ ]:
new_column_name = 'HighPercentEdit'
df_train[new_column_name] = ''

#Getting all unique symbols
symbols = df_train['Symbol'].unique()
for symbol in symbols:
    #Getting all rows for particular symbol
    symbol_data = df_train[df_train['Symbol'] == symbol]
    print(f"Running for symbol ;{symbol}")

    #Getting all the dates for which the data is present for that symbol
    unique_dates = symbol_data['Date'].unique()
    for date in unique_dates:
        #Getting all the rows for particular date
        data_for_date = symbol_data[symbol_data['Date'] == date]
        print(f"Running for date: {date}")

#         print(type(data_for_date))
        value = data_for_date.iloc[-1]['High5minPercent']
        print(f"Adding value: {value}")
        for i,r in data_for_date.iterrows():
            df_train.at[i,new_column_name] = value

#         break

#     break


Streaming output truncated to the last 5000 lines.
Running for date: 2016-03-09T00:00:00.000000000
Adding value: -0.27
Running for date: 2016-11-04T00:00:00.000000000
Adding value: 1.95
Running for date: 2017-03-09T00:00:00.000000000
Adding value: -1.8
Running for date: 2017-05-10T00:00:00.000000000
Adding value: 1.2
Running for date: 2017-11-08T00:00:00.000000000
Adding value: -9.52
Running for date: 2018-03-08T00:00:00.000000000
Adding value: 2.33
Running for date: 2018-05-09T00:00:00.000000000
Adding value: -0.83
Running for date: 2018-08-08T00:00:00.000000000
Adding value: 8.54
Running for date: 2018-11-07T00:00:00.000000000
Adding value: 1.81
Running for date: 2019-03-07T00:00:00.000000000
Adding value: 2.05
Running for date: 2019-05-08T00:00:00.000000000
Adding value: -1.37
Running for symbol ;BCLI
Running for date: 2016-03-10T00:00:00.000000000
Adding value: 1.65
Running for date: 2016-05-10T00:00:00.000000000
Adding value: -0.47
Running for date: 2016-08-11T00:00:00.000000000
A

In [ ]:
new_column_name = 'VolumePercentEdit'
df_train[new_column_name] = ''

#Getting all unique symbols
symbols = df_train['Symbol'].unique()
for symbol in symbols:
    #Getting all rows for particular symbol
    symbol_data = df_train[df_train['Symbol'] == symbol]
    print(f"Running for symbol ;{symbol}")

    #Getting all the dates for which the data is present for that symbol
    unique_dates = symbol_data['Date'].unique()
    for date in unique_dates:
        #Getting all the rows for particular date
        data_for_date = symbol_data[symbol_data['Date'] == date]
        print(f"Running for date: {date}")

#         print(type(data_for_date))
        value = data_for_date.iloc[-1]['Vol5minPercent']
        print(f"Adding value: {value}")
        for i,r in data_for_date.iterrows():
            df_train.at[i,new_column_name] = value

#         break

#     break


Streaming output truncated to the last 5000 lines.
Running for date: 2016-03-09T00:00:00.000000000
Adding value: 62.5
Running for date: 2016-11-04T00:00:00.000000000
Adding value: 16.9
Running for date: 2017-03-09T00:00:00.000000000
Adding value: 357.2
Running for date: 2017-05-10T00:00:00.000000000
Adding value: 247.3
Running for date: 2017-11-08T00:00:00.000000000
Adding value: 1102.2
Running for date: 2018-03-08T00:00:00.000000000
Adding value: 467.3
Running for date: 2018-05-09T00:00:00.000000000
Adding value: 78.5
Running for date: 2018-08-08T00:00:00.000000000
Adding value: 667.1
Running for date: 2018-11-07T00:00:00.000000000
Adding value: 135.9
Running for date: 2019-03-07T00:00:00.000000000
Adding value: 306.8
Running for date: 2019-05-08T00:00:00.000000000
Adding value: 129.9
Running for symbol ;BCLI
Running for date: 2016-03-10T00:00:00.000000000
Adding value: 508.1
Running for date: 2016-05-10T00:00:00.000000000
Adding value: 50.7
Running for date: 2016-08-11T00:00:00.00000

In [ ]:
df_train['time_idx'] = df_train['time_idx'].astype('int')
df_train[['year','month', 'day', 'time_idx']] = df_train[['year','month', 'day', 'time_idx']].astype(int)
df_train[['year','month', 'day','hour']] = df_train[['year','month', 'day', 'hour']].astype(str)
#df_train[['Guidance']] = df_train[['Guidance']].astype(str)
df_train['FinalTime'] = df_train['FinalTime'].astype('float64')
df_train['HighPercentEdit'] = df_train['HighPercentEdit'].astype('float64')
df_train['VolumePercentEdit'] = df_train['VolumePercentEdit'].astype('float64')
#df_train['Revenue'] = df_train['Revenue'].astype('float64')
#df_train[['Target2Percent','Target1Percent']] = df_train[['Target2Percent','Target1Percent']].astype(str)
#df_train[['Long_Target']] = df_train[['Long_Target']].astype(str)

print(df_train.dtypes)

Symbol                        object
Date                  datetime64[ns]
Time                  datetime64[ns]
High5minPercent              float64
Low5minPercent               float64
Close5minPercent             float64
Vol5minPercent               float64
ChangePercent                float64
PreviousClosePrice           float64
PrevChangePercent            float64
Volatility                   float64
EstimatedEPS                 float64
EPS                          float64
SymbolName_Sum               float64
CompanyName_Sum              float64
OpenNBIPercent               float64
Current_price                float64
Velocity                     float64
Acceleration                 float64
time_idx                       int64
year                          object
month                         object
day                           object
hour                          object
FinalTime                    float64
HighPercentEdit              float64
VolumePercentEdit            float64
d

In [ ]:
index_drop11 = df_train[(df_train['FinalTime'] >= 30)|(df_train['FinalTime'] <= -30)].index
df_train.drop(index_drop11, inplace=True)
index_drop12 = df_train[(df_train['HighPercentEdit'] >= 30)|(df_train['HighPercentEdit'] <= -30)].index
df_train.drop(index_drop12, inplace=True)
index_drop13 = df_train[(df_train['VolumePercentEdit'] < 150)].index
df_train.drop(index_drop13, inplace=True)

#index_drop13 = df_train[(df_train['Guidance'] == 0.0)].index
#df_train.drop(index_drop13, inplace=True)

df_train.describe()

,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,Current_price,Velocity,Acceleration,time_idx,FinalTime,HighPercentEdit,VolumePercentEdit
count,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000
mean,0.585177,-1.144226,-0.293093,536.187313,-0.015602,40.003209,0.103075,49.581560,0.071148,0.101731,363.210572,113.962909,-0.009007,39.997594,0.000156,-0.000163,4.500000,-0.275266,0.214352,516.652862
std,7.058598,6.989953,7.053303,451.437189,8.189397,46.403372,3.441085,24.380096,0.894122,0.986797,357.261220,206.640774,0.738806,46.682869,0.560410,0.799823,2.872323,7.321855,7.340952,408.951686
min,-30.560000,-35.290000,-35.290000,0.000000,-24.824400,2.019000,-18.225900,10.196500,-2.910000,-8.000000,0.000000,0.000000,-3.437798,1.549944,-12.312480,-14.058304,0.000000,-27.430000,-25.770000,150.100000
25%,-3.450000,-5.090000,-4.350000,239.400000,-4.955850,8.350000,-1.567975,32.584375,-0.420000,-0.410000,90.750000,0.000000,-0.352107,8.194741,-0.089900,-0.140250,2.000000,-4.582500,-4.152500,244.750000
50%,0.380000,-0.900000,-0.240000,375.900000,0.000000,20.145000,0.000000,46.386400,-0.110000,-0.080000,237.000000,31.000000,0.000970,20.270888,0.000000,0.000000,4.500000,-0.240000,0.105000,369.300000
75%,4.560000,3.000000,3.850000,657.800000,4.983375,53.662500,1.582850,61.838800,0.430000,0.470000,532.250000,123.000000,0.320124,53.831687,0.093363,0.140155,7.000000,4.180000,4.620000,629.250000
max,52.750000,31.370000,33.530000,2000.000000,24.764200,248.580000,28.191500,242.752400,7.530000,8.000000,1570.000000,1546.000000,6.262782,267.101030,12.056148,18.056688,9.000000,26.440000,26.890000,2000.000000


# NOW CREATE CLASSIFICATIONS AND WEIGHTING BEFORE BUILDING PYTORCH FORECASTING MODEL

In [ ]:
#Classifier Targets
LongStrategy = 0
conditionT = [
  (df_train['FinalTime'] <= 0),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] <= LongStrategy),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] > LongStrategy)
]
valueW = [0, 0, 1]
df_train['Target'] = np.select(conditionT, valueW)

df_train.describe()

total = df_train['Target'].sum()
print(total)
df_train.describe()

NameError: ignored

In [ ]:
#weighting LONG STRATEGY and SHORT STRATEGY ()
LongStrategy = 2
ShortStrategy = -2
conditionW = [
  (df_train['FinalTime'] <= 0) & (df_train['ChangePercent'] - df_train['FinalTime'] <= ShortStrategy),
  (df_train['FinalTime'] <= 0) & (df_train['ChangePercent'] - df_train['FinalTime'] > ShortStrategy),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] <= LongStrategy),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] > LongStrategy)
]
valueW = [0.5, 0.25, 0.25, 0.5]
df_train['weight'] = np.select(conditionW, valueW)

df_train.describe()

In [ ]:
#weighting LONG STRATEGY
LongStrategy = 2
conditionW = [
  (df_train['FinalTime'] <= 0),
  #(df_train['FinalTime'] > 0),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] <= LongStrategy),
  (df_train['FinalTime'] > 0) & (df_train['ChangePercent'] - df_train['FinalTime'] > LongStrategy)
]
valueW = [0.25, 0.25, 0.5]
df_train['weight'] = np.select(conditionW, valueW)

df_train.describe()

,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,...,CompanyName_Sum,OpenNBIPercent,Current_price,Velocity,Acceleration,time_idx,FinalTime,HighPercentEdit,VolumePercentEdit,weight
count,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,...,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000
mean,0.585177,-1.144226,-0.293093,536.187313,-0.015602,40.003209,0.103075,49.581560,0.071148,0.101731,...,113.962909,-0.009007,39.997594,0.000156,-0.000163,4.500000,-0.275266,0.214352,516.652862,0.283148
std,7.058598,6.989953,7.053303,451.437189,8.189397,46.403372,3.441085,24.380096,0.894122,0.986797,...,206.640774,0.738806,46.682869,0.560410,0.799823,2.872323,7.321855,7.340952,408.951686,0.084785
min,-30.560000,-35.290000,-35.290000,0.000000,-24.824400,2.019000,-18.225900,10.196500,-2.910000,-8.000000,...,0.000000,-3.437798,1.549944,-12.312480,-14.058304,0.000000,-27.430000,-25.770000,150.100000,0.250000
25%,-3.450000,-5.090000,-4.350000,239.400000,-4.955850,8.350000,-1.567975,32.584375,-0.420000,-0.410000,...,0.000000,-0.352107,8.194741,-0.089900,-0.140250,2.000000,-4.582500,-4.152500,244.750000,0.250000
50%,0.380000,-0.900000,-0.240000,375.900000,0.000000,20.145000,0.000000,46.386400,-0.110000,-0.080000,...,31.000000,0.000970,20.270888,0.000000,0.000000,4.500000,-0.240000,0.105000,369.300000,0.250000
75%,4.560000,3.000000,3.850000,657.800000,4.983375,53.662500,1.582850,61.838800,0.430000,0.470000,...,123.000000,0.320124,53.831687,0.093363,0.140155,7.000000,4.180000,4.620000,629.250000,0.250000
max,52.750000,31.370000,33.530000,2000.000000,24.764200,248.580000,28.191500,242.752400,7.530000,8.000000,...,1546.000000,6.262782,267.101030,12.056148,18.056688,9.000000,26.440000,26.890000,2000.000000,0.500000


In [ ]:
df_train.dtypes

Symbol                        object
Date                  datetime64[ns]
Time                  datetime64[ns]
High5minPercent              float64
Low5minPercent               float64
Close5minPercent             float64
Vol5minPercent               float64
ChangePercent                float64
PreviousClosePrice           float64
PrevChangePercent            float64
Volatility                   float64
EstimatedEPS                 float64
EPS                          float64
SymbolName_Sum               float64
CompanyName_Sum              float64
OpenNBIPercent               float64
Current_price                float64
Velocity                     float64
Acceleration                 float64
time_idx                       int64
year                          object
month                         object
day                           object
hour                          object
FinalTime                    float64
HighPercentEdit              float64
VolumePercentEdit            float64
w

In [ ]:
#edited version for Pytrends
max_prediction_length = 1
max_encoder_length = 10 #for 11am, change to

import datetime
training_cutoff = datetime.datetime(2018, 9, 1)

training = TimeSeriesDataSet(
    df_train[lambda x: x.Date < training_cutoff],
    time_idx='time_idx',
    target="ChangePercent",
    group_ids=['Symbol','year', 'month', 'day'],
    weight = 'weight',
    min_encoder_length=0,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['Symbol'],
    static_reals=['PreviousClosePrice','PrevChangePercent', 'Volatility',	'EstimatedEPS',	'EPS'],
    time_varying_known_categoricals=['year', 'month', 'hour', 'day'],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    #time_varying_unknown_reals=['Close5minPercent','High5minPercent', 'Low5minPercent','Vol5minPercent', 'Acceleration'],
    time_varying_unknown_reals=['High5minPercent','Low5minPercent','Close5minPercent','Vol5minPercent'],
    categorical_encoders={'Symbol': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'year': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'month': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'day': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True) },
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [ ]:
max_prediction_length = 1
max_encoder_length = 10 #for 11am, change to

import datetime
training_cutoff = datetime.datetime(2018, 9, 1)

training = TimeSeriesDataSet(
   df_train[lambda x: x.Date < training_cutoff],
    time_idx='time_idx',
    target="ChangePercent",
    group_ids=['Symbol','year', 'month', 'day', 'Guidance'],
    weight = 'weight',
    min_encoder_length=0,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['Symbol', 'Guidance'],
    static_reals=['PreviousClosePrice','PrevChangePercent', 'Volatility',	'EstimatedEPS',	'EPS', 'OpenNBIPercent'],
    time_varying_known_categoricals=['year', 'month', 'hour', 'day'],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    #time_varying_unknown_reals=['Close5minPercent','High5minPercent', 'Low5minPercent','Vol5minPercent', 'Acceleration'],
    time_varying_unknown_reals=['High5minPercent','Low5minPercent','Close5minPercent','Vol5minPercent'],
    categorical_encoders={'Symbol': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),'Guidance': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'year': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'month': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'day': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True) },
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

TypeError: ignored

In [ ]:
#MultiLoss Code
from pytorch_forecasting.data.encoders import EncoderNormalizer, MultiNormalizer, TorchNormalizer, NaNLabelEncoder

max_prediction_length = 1
max_encoder_length = 10 #for 11am, change to

import datetime
training_cutoff = datetime.datetime(2018, 9, 1)

training = TimeSeriesDataSet(
    df_train[lambda x: x.Date < training_cutoff],
    time_idx='time_idx',
    target=['Target','ChangePercent'],
    group_ids=['Symbol','year', 'month', 'day'],
    weight = 'weight',
    min_encoder_length=0,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=['Symbol'],
    static_reals=['PreviousClosePrice','PrevChangePercent', 'Volatility',	'EstimatedEPS',	'EPS', 'OpenNBIPercent'],
    time_varying_known_categoricals=['year', 'month', 'hour', 'day'],
    time_varying_known_reals=[],
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=['Close5minPercent','High5minPercent', 'Low5minPercent','Vol5minPercent'],
    categorical_encoders={'Symbol': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),'year': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'month': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True), 'day': pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True) },
    target_normalizer = MultiNormalizer([NaNLabelEncoder(),TorchNormalizer()]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True
)

In [ ]:
validation = TimeSeriesDataSet.from_dataset(training,  df_train[lambda x: x.Date > training_cutoff], predict=True, stop_randomization=True)

batch_size = 32
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)

In [ ]:
# configure network and trainer
pl.seed_everything(42)
trainer = pl.Trainer(
    #gpus=0,
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=0.003,
    hidden_size=16,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=4,
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=7,  # 7 quantiles by default
    #loss=CrossEntropy(),
    #loss=MultiLoss(metrics=[CrossEntropy(),QuantileLoss()], weights=[1.0,2.0]),
    loss=QuantileLoss(),
    # reduce learning rate if no improvement in validation loss after x epochs
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

INFO: Global seed set to 42
INFO:lightning.fabric.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Number of parameters in network: 31.4k


In [ ]:
# find optimal learning rate
#tuner = pl.tuner.Tuner(trainer)


res = tuner.lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=1.,
    min_lr=1e-10,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

NameError: ignored

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard
#from pytorch_lightning.loggers import WandbLogger
#wandb_logger = WandbLogger()
#logger=wandb_logger


trainer = pl.Trainer(
    max_epochs=40,
   #gpus=0,
    #weights_summary="top",
    gradient_clip_val=0.171,
    limit_train_batches=30,  # comment in for training, running valiation every 30 batches
    # fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.003,
    hidden_size=58,
    attention_head_size=4,
    dropout=0.146,
    hidden_continuous_size=31,
    #output_size=7,  # 7 quantiles by default
    #loss=CrossEntropy(),
    loss=QuantileLoss(),
    output_size=7,  # 7 quantiles by default
    #loss=MultiLoss(metrics=[CrossEntropy(),QuantileLoss()], weights=[1.0,2.0]),
    log_interval=0,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Number of parameters in network: 283.9k


In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

INFO: 
   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 21.1 K
3  | prescalers                         | ModuleDict                      | 806   
4  | static_variable_selection          | VariableSelectionNetwork        | 51.5 K
5  | encoder_variable_selection         | VariableSelectionNetwork        | 33.4 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 7.4 K 
7  | static_context_variable_selection  | GatedResidualNetwork            | 13.8 K
8  | static_context_initial_hidden_lstm | GatedResidualNetwork            | 13.8 K
9  | static_context_initial_cell_lstm   | GatedResidualNetwork            

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=40` reached.


In [ ]:
wandb.finish()

In [ ]:
import pickle

from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=50,
    max_epochs=40,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 64),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(2, 4),
    learning_rate_range=(0.0001, 0.1),
    dropout_range=(0.1, 0.2),
    trainer_kwargs=dict(limit_train_batches=32),
    reduce_on_plateau_patience=10,
    use_learning_rate_finder=True,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

In [ ]:
%cd /content/gdrive/My\ Drive/Forecaster/GooglePytrends/lightning_logs/default/version_14/checkpoints
#%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_PrevCloseData/Earnings_PrevDayData_GuidanceFlag_5-20-22_v3/35u21krr/checkpoints
#%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_Jan22/Regression/lightning_logs/default/version_3/checkpoints/optuna_test/trial_29
#%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_PrevCloseData/lightning_logs/default/version_0/checkpoints
!ls

/content/gdrive/My Drive/Forecaster/GooglePytrends/lightning_logs/default/version_14/checkpoints
 Biotech_Earnings_TestDummyChangeValue_v1.csv
'Copy of Earnings_TEST_AllData_v1.csv'
 Earnings_TEST_AllData_v1.csv
 earnings_v5-20-22_pmi2mj2o.csv
 Earnings_ValDataloader_Prediction.csv
'Earnings_ValDataloader_Prediction_dummy-values_Change&weight.csv'
 Earnings_ValDataloader_Prediction_dummy-values.csv
 Earnings_ValDataloader_Prediction_dummy-values_VolatilityChange.csv
'epoch=32-step=989.ckpt'
'epoch=39-step=1199.ckpt'
 lightning_logs
 One_Stock_Test_v1.csv
 OneStock_v1.csv
 Predictions_AllData_v1.csv
 Predictions_AllSymbols_ZeroChangePercent.csv
 Predictions_AllSymbols_ZeroChangePercent.gsheet
 Predictions_OneStock_v1.csv


In [ ]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)

#best_model_path = trainer.checkpoint_callback.best_model_path
#best_model_path = "/content/gdrive/My\ Drive/Forecaster/Volatility/lightning_logs/default/version_0/checkpoints/epoch=9-step=299.ckpt"

best_tft = TemporalFusionTransformer.load_from_checkpoint("epoch=32-step=989.ckpt")
#best_tft = TemporalFusionTransformer.load_from_checkpoint("epoch=47.ckpt")


In [ ]:
df_train.describe()

,Unnamed: 0,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,time_idx,FinalTime,HighPercentEdit,VolumePercentEdit,weight
count,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000,34240.000000
mean,25539.853388,0.585177,-1.144226,-0.293093,536.187313,-0.015602,40.003209,0.103075,49.581560,0.071148,0.101731,363.210572,113.962909,-0.009007,4.500000,-0.275266,0.214352,516.652862,0.283148
std,15237.560619,7.058598,6.989953,7.053303,451.437189,8.189397,46.403372,3.441085,24.380096,0.894122,0.986797,357.261220,206.640774,0.738806,2.872323,7.321855,7.340952,408.951686,0.084785
min,0.000000,-30.560000,-35.290000,-35.290000,0.000000,-24.824400,2.019000,-18.225900,10.196500,-2.910000,-8.000000,0.000000,0.000000,-3.437798,0.000000,-27.430000,-25.770000,150.100000,0.250000
25%,12499.750000,-3.450000,-5.090000,-4.350000,239.400000,-4.955850,8.350000,-1.567975,32.584375,-0.420000,-0.410000,90.750000,0.000000,-0.352107,2.000000,-4.582500,-4.152500,244.750000,0.250000
50%,24954.500000,0.380000,-0.900000,-0.240000,375.900000,0.000000,20.145000,0.000000,46.386400,-0.110000,-0.080000,237.000000,31.000000,0.000970,4.500000,-0.240000,0.105000,369.300000,0.250000
75%,38649.250000,4.560000,3.000000,3.850000,657.800000,4.983375,53.662500,1.582850,61.838800,0.430000,0.470000,532.250000,123.000000,0.320124,7.000000,4.180000,4.620000,629.250000,0.250000
max,53029.000000,52.750000,31.370000,33.530000,2000.000000,24.764200,248.580000,28.191500,242.752400,7.530000,8.000000,1570.000000,1546.000000,6.262782,9.000000,26.440000,26.890000,2000.000000,0.500000


## SINGLE LOSS ANALYSIS


In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions, x = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
quantile_predict, x = best_tft.predict(val_dataloader, mode="quantiles", return_x=True)

In [ ]:
print(quantile_predict[:, :, 4:5])
quantile= quantile_predict[:, :, 4:5]

tensor([[[-4.6272e+00]],

        [[ 1.4033e+01]],

        [[ 1.0881e+01]],

        [[ 1.0982e+01]],

        [[ 5.7990e+00]],

        [[ 2.2329e+00]],

        [[ 5.0536e+00]],

        [[ 9.9685e+00]],

        [[ 1.8851e+01]],

        [[ 1.6884e+01]],

        [[-5.1711e+00]],

        [[ 2.4884e+00]],

        [[-3.6907e+00]],

        [[-8.7680e+00]],

        [[-4.7823e+00]],

        [[-3.4063e+00]],

        [[ 4.6760e+00]],

        [[ 5.1735e-02]],

        [[ 2.2558e+00]],

        [[ 1.1072e+01]],

        [[-2.7091e+00]],

        [[-4.5240e+00]],

        [[-6.4843e+00]],

        [[-1.4153e+01]],

        [[ 6.5394e+00]],

        [[ 1.1008e+01]],

        [[ 1.3245e+01]],

        [[ 7.2733e+00]],

        [[-5.5564e+00]],

        [[ 6.8492e+00]],

        [[ 7.5520e+00]],

        [[ 7.2353e+00]],

        [[-2.1941e+00]],

        [[-2.3991e+00]],

        [[-3.2789e+00]],

        [[ 7.8961e+00]],

        [[ 6.9185e+00]],

        [[ 2.0122e+01]],

        [[ 6

In [ ]:
quantile_predict

tensor([[[-10.6499,  -8.5943,  -7.4878,  ...,  -4.6272,  -2.4408,   0.4167]],

        [[  4.1552,   8.2339,  10.9190,  ...,  14.0325,  13.9842,  16.6500]],

        [[  0.4180,   4.2954,   6.9165,  ...,  10.8813,  11.6424,  15.6812]],

        ...,

        [[-15.0552,  -9.7057,  -6.5375,  ...,   1.5152,   6.0567,  11.1734]],

        [[ -8.4376,  -3.1199,   0.1875,  ...,   7.6614,  11.4082,  16.3355]],

        [[ -6.3020,  -0.9357,   2.3191,  ...,   8.9963,  11.2949,  15.8510]]])

In [ ]:
df_train_predict = df_train[lambda x: x.Date > training_cutoff]
#df_train_predict = df_train
df_train_predict = df_train_predict[df_train_predict['time_idx'] == 9]
df_train_predict.describe()

,Unnamed: 0,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,PrevChangePercent,Volatility,EstimatedEPS,EPS,SymbolName_Sum,CompanyName_Sum,OpenNBIPercent,time_idx,FinalTime,HighPercentEdit,VolumePercentEdit,weight
count,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.000000,857.0,857.000000,857.000000,857.000000,857.000000
mean,45644.215869,0.347106,-0.739685,-0.186289,481.670012,0.190777,41.737185,0.268043,52.891639,0.023302,0.073827,348.068845,122.927655,-0.093176,9.0,-0.186289,0.347106,481.670012,0.285589
std,4154.982425,7.459103,7.328146,7.414041,383.040191,8.579897,48.381003,3.473038,23.333181,0.929296,1.027225,349.295387,216.672778,0.593440,0.0,7.414041,7.459103,383.040191,0.087405
min,38649.000000,-23.200000,-24.470000,-24.430000,150.700000,-24.824400,2.032900,-15.024000,10.196500,-2.910000,-5.870000,0.000000,0.000000,-2.527416,9.0,-24.430000,-23.200000,150.700000,0.250000
25%,42029.000000,-3.810000,-4.650000,-4.230000,240.100000,-4.753000,8.170000,-1.596900,35.775100,-0.480000,-0.460000,85.000000,0.000000,-0.518227,9.0,-4.230000,-3.810000,240.100000,0.250000
50%,45419.000000,0.460000,-0.540000,0.080000,345.300000,0.502500,20.890000,0.210100,50.063200,-0.150000,-0.110000,214.000000,36.000000,-0.070622,9.0,0.080000,0.460000,345.300000,0.250000
75%,49049.000000,4.800000,3.650000,4.380000,562.300000,5.485100,58.660000,1.751300,65.254700,0.420000,0.470000,520.000000,135.000000,0.342648,9.0,4.380000,4.800000,562.300000,0.250000
max,53029.000000,24.020000,21.650000,23.050000,2000.000000,24.764200,247.160000,20.031800,227.701600,7.530000,8.000000,1570.000000,1546.000000,1.723811,9.0,23.050000,24.020000,2000.000000,0.500000


In [ ]:
#Prepare output dataframe & file with predictions and original data
predictions,X,index,decoder_lengths = best_tft.predict(val_dataloader, mode="prediction", return_x=True, return_decoder_lengths=True)

ValueError: ignored

In [ ]:
predictions

NameError: ignored

In [ ]:
index.head()

NameError: ignored

In [ ]:
#predict_df = pd.DataFrame(predictions).astype("float")
predict_df = pd.DataFrame(quantile).astype("float")
predict_df.columns = ['Prediction']
predict_df.head()

,Prediction
0,1.066734e-08
1,-2.212579e-09
2,-5.508248e-09
3,7.691208e-09
4,7.764315e-09


In [ ]:
results_df = pd.concat([index, predict_df], axis=1)
results_df.describe()

,time_idx,Prediction
count,3424.0,3.424000e+03
mean,9.0,3.533063e-09
std,0.0,9.612111e-09
min,9.0,-1.876766e-08
25%,9.0,-2.949005e-09
50%,9.0,3.218964e-09
75%,9.0,1.027659e-08
max,9.0,2.671900e-08


In [ ]:
results_df = results_df.sort_values(["Symbol", "year", "month"], ascending = (True, True, True))
#final_results_df = pd.concat([df_train_predict,results_df], axis=1)
final_df = df_train_predict.merge(results_df, how = 'inner', on = ['Symbol', 'year', 'month', 'day'])
#final_df.to_csv('earnings_regression_updateweights_optunaparams_2-8-22_trial_22.csv')
final_df.head()

,Unnamed: 0,Symbol,Date,Time,High5minPercent,Low5minPercent,Close5minPercent,Vol5minPercent,ChangePercent,PreviousClosePrice,...,year,month,day,hour,FinalTime,HighPercentEdit,VolumePercentEdit,weight,time_idx_y,Prediction
0,9,WBA,2016-01-07,2022-12-11 10:15:00,1.34,0.80,1.04,321.0,0.0,79.6300,...,2016,1,7,10,1.04,1.34,321.0,0.25,9,3.505838e-09
1,19,ANGO,2016-01-08,2022-12-11 10:15:00,-4.24,-4.59,-4.42,405.1,0.0,11.5500,...,2016,1,8,10,-4.42,-4.24,405.1,0.25,9,-2.196799e-09
2,29,UNH,2016-01-19,2022-12-11 10:15:00,3.38,3.04,3.18,203.2,0.0,109.2700,...,2016,1,19,10,3.18,3.38,203.2,0.25,9,5.729157e-09
3,49,ISRG,2016-01-22,2022-12-11 10:15:00,1.62,0.88,1.44,350.2,0.0,185.1815,...,2016,1,22,10,1.44,1.62,350.2,0.25,9,4.074681e-09
4,59,RMD,2016-01-22,2022-12-11 10:15:00,10.86,9.20,9.31,729.6,0.0,54.0400,...,2016,1,22,10,9.31,10.86,729.6,0.25,9,1.139154e-08


In [ ]:
#ideally change the date and append to end of filename for future reference
final_df.to_csv('Predictions_AllSymbols_ZeroChangePercent.csv')

In [ ]:
#Calculate Profitability of Predictions
#Long Strategy
Long = 4.75
VolPercent= 200

condition1 = [
  (final_df['Close5minPercent'] > 0) & (final_df['Prediction'] - final_df['Close5minPercent'] > Long) & (final_df['Vol5minPercent']>VolPercent)
]
value1 = [final_df['ChangePercent'] - final_df['Close5minPercent']]
final_df['Trade_Long'] = np.select(condition1, value1)

#Short Strategy
Short = -3.5
Short_Vol = 200
condition2 = [
   (final_df['Close5minPercent'] < 0) & (final_df['Prediction'] - final_df['Close5minPercent'] < Short) & (final_df['Vol5minPercent']> Short_Vol)
]
value2 = [-(final_df['ChangePercent'] - final_df['Close5minPercent'])]
final_df['Trade_Short'] = np.select(condition2, value2)

final_df.to_csv('earnings_v5-20-22_pmi2mj2o.csv')
final_df.head(50)

In [ ]:
#Long
Trade_Short_TotalProfit = final_df[final_df.Trade_Short != 0].sum()["Trade_Short"]
Trade_Short_Trades = final_df[(final_df['Close5minPercent'] < 0) & (final_df['Prediction'] - final_df['Close5minPercent'] < Short)].count()["Trade_Short"]
Trade_Short_Avg = (Trade_Short_TotalProfit/Trade_Short_Trades)
Trade_Short_Win = (final_df[final_df.Trade_Short >0].count()["Trade_Short"])/Trade_Short_Trades

print("Short_Profit = ",Trade_Short_TotalProfit, "  Short#Trades = ", Trade_Short_Trades, "   Avg % Profit =", np.round(Trade_Short_Avg, decimals=2), "   Win % =", np.round(Trade_Short_Win, decimals=2) )

#Short
Trade_Long_TotalProfit = final_df[final_df.Trade_Long != 0].sum()["Trade_Long"]
Trade_Long_Trades = final_df[(final_df['Close5minPercent'] > 0) & (final_df['Prediction'] - final_df['Close5minPercent'] > Long) & (final_df['Vol5minPercent'] > VolPercent)].count()["Trade_Long"]
Trade_Long_Avg = (Trade_Long_TotalProfit/Trade_Long_Trades)
Trade_Long_Win = (final_df[final_df.Trade_Long >0].count()["Trade_Long"])/Trade_Long_Trades

print("Long_Profit = ",Trade_Long_TotalProfit, "   Long#Trades = ", Trade_Long_Trades, "   Avg % Profit =", np.round(Trade_Long_Avg, decimals=2), "   Win % =", np.round(Trade_Long_Win, decimals=2) )

## MultiLoss Model Profitability Analysis

In [ ]:
#Prepare output dataframe & file with predictions and original data
predictions,X,index,decoder_lengths = best_tft.predict(val_dataloader, mode="prediction", return_x=True, return_index = True, return_decoder_lengths=True)

In [ ]:
TwoTensors = torch.cat(predictions, dim=1)
predict_df = pd.DataFrame(TwoTensors).astype("float")
predict_df.describe()

In [ ]:
#multiloss methodology
results_df = pd.concat([index, predict_df], axis=1)
results_df.describe()
#results_df.to_csv('biotech_earnings_4-9-22_QuantileLoss.csv')

In [ ]:
df_train_predict = df_train[lambda x: x.Date > training_cutoff]

#df_train_predict = df_train
df_train_predict = df_train_predict[df_train_predict['time_idx'] == 9]
df_train_predict.head()

In [ ]:
results_df = results_df.sort_values(["Symbol", "year", "month"], ascending = (True, True, True))
#final_results_df = pd.concat([df_train_predict,results_df], axis=1)
final_df = df_train_predict.merge(results_df, how = 'inner', on = ['Symbol', 'year', 'month', 'day'])
#final_df.to_csv('earnings_regression_updateweights_optunaparams_2-8-22_trial_22.csv')
final_df.head()

In [ ]:
results_df = results_df.sort_values(["Symbol", "year", "month"], ascending = (True, True, True))
#final_results_df = pd.concat([df_train_predict,results_df], axis=1)
final_df = df_train_predict.merge(results_df, how = 'inner', on = ['Symbol', 'year', 'month', 'day'])
final_df.to_csv('earnings_prevchange_v1.csv')
final_df.head()

In [ ]:
#Calculate Profitability of Predictions
#Long Strategy
Long = 1
condition1 = [
  (final_df['Close5minPercent'] > 0) & (final_df[1] - final_df['Close5minPercent'] > Long)
]
value1 = [final_df['ChangePercent'] - final_df['Close5minPercent']]
final_df['Trade_Long'] = np.select(condition1, value1)

#Short Strategy
Short = -1
condition2 = [
   (final_df['Close5minPercent'] < 0) & (final_df[1] - final_df['Close5minPercent'] < Short)
]
value2 = [-(final_df['ChangePercent'] - final_df['Close5minPercent'])]
final_df['Trade_Short'] = np.select(condition2, value2)

final_df.to_csv('earnings_v5-20-22_v1.csv')
final_df.head(50)

In [ ]:
#Long
Trade_Short_TotalProfit = final_df[final_df.Trade_Short != 0].sum()["Trade_Short"]
Trade_Short_Trades = final_df[(final_df['Close5minPercent'] < 0) & (final_df[1] - final_df['Close5minPercent'] < Short)].count()["Trade_Short"]
Trade_Short_Avg = (Trade_Short_TotalProfit/Trade_Short_Trades)
Trade_Short_Win = (final_df[final_df.Trade_Short >0].count()["Trade_Short"])/Trade_Short_Trades

print("Short_Profit = ",Trade_Short_TotalProfit, "  Short#Trades = ", Trade_Short_Trades, "   Avg % Profit =", np.round(Trade_Short_Avg, decimals=2), "   Win % =", np.round(Trade_Short_Win, decimals=2) )

#Short
Trade_Long_TotalProfit = final_df[final_df.Trade_Long != 0].sum()["Trade_Long"]
Trade_Long_Trades = final_df[(final_df['Close5minPercent'] > 0) & (final_df[1] - final_df['Close5minPercent'] > Long)].count()["Trade_Long"]
Trade_Long_Avg = (Trade_Long_TotalProfit/Trade_Long_Trades)
Trade_Long_Win = (final_df[final_df.Trade_Long >0].count()["Trade_Long"])/Trade_Long_Trades

print("Long_Profit = ",Trade_Long_TotalProfit, "   Long#Trades = ", Trade_Long_Trades, "   Avg % Profit =", np.round(Trade_Long_Avg, decimals=2), "   Win % =", np.round(Trade_Long_Win, decimals=2) )

In [ ]:
df_prediction = training.decoded_index
df_prediction.head()

In [ ]:
#Actuals vs Predictions
predictions, x = best_tft.predict(val_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(x, predictions)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals)

interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

In [ ]:
#variable importance
interpretation = best_tft.interpret_output(raw_predictions, reduction="sum")
best_tft.plot_interpretation(interpretation)

# **OPTUNA VISUALIZATION**

In [ ]:
%matplotlib inline

import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna

pd.options.display.max_columns = 100

In [ ]:
%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_Jan22/Regression/lightning_logs/default/version_3/checkpoints
!ls

/content/gdrive/My Drive/Forecaster/StockData/Earnings_Jan22/Regression/lightning_logs/default/version_3/checkpoints
 earnings_regression_weights_2-3-22_v1.csv   lightning_logs
 earnings_regression_weights_2-3-22_v2.csv   optuna_test
'epoch=49-step=1499.ckpt'		     test_study.pkl


In [ ]:
data = joblib.load('test_study2.pkl')

df_optuna = data.trials_dataframe()
df_optuna.dropna(inplace=True)
df_optuna.reset_index(inplace=True)

df_optuna['time'] = df_optuna.datetime_complete - df_optuna.datetime_start
df_optuna['time'] = df_optuna.time.astype('int') / (1_000_000_000)
df_optuna = df_optuna[df_optuna.time>0]

In [ ]:
df_optuna.head(30)

,index,number,value,datetime_start,datetime_complete,duration,params_attention_head_size,params_dropout,params_gradient_clip_val,params_hidden_continuous_size,params_hidden_size,params_learning_rate,system_attrs_completed_rung_0,system_attrs_completed_rung_1,system_attrs_completed_rung_2,state,time
0,1,1,0.265719,2022-06-20 17:01:52.239172,2022-06-20 17:05:35.501327,0 days 00:03:43.262155,3,0.104851,0.068686,12,52,0.016596,0.394736,0.305435,0.277539,COMPLETE,223.262155
1,24,24,0.284210,2022-06-20 17:15:31.397196,2022-06-20 17:17:12.546683,0 days 00:01:41.149487,2,0.117930,0.166391,18,34,0.016596,0.404532,0.304679,0.284210,PRUNED,101.149487
2,33,33,0.291342,2022-06-20 17:20:59.228663,2022-06-20 17:22:40.733510,0 days 00:01:41.504847,2,0.106491,0.302161,18,64,0.016596,0.372124,0.304642,0.291342,PRUNED,101.504847
3,36,36,0.282200,2022-06-20 17:23:32.079920,2022-06-20 17:25:15.280960,0 days 00:01:43.201040,3,0.105342,0.369584,50,64,0.015488,0.368152,0.301177,0.282200,PRUNED,103.201040
4,41,41,0.290084,2022-06-20 17:27:26.075756,2022-06-20 17:29:12.478466,0 days 00:01:46.402710,3,0.109967,0.361966,61,64,0.014454,0.346144,0.298979,0.290084,PRUNED,106.402710
5,44,44,0.291319,2022-06-20 17:30:20.339381,2022-06-20 17:32:07.326144,0 days 00:01:46.986763,3,0.109875,0.198710,62,64,0.016596,0.390807,0.302134,0.291319,PRUNED,106.986763
6,46,46,0.285095,2022-06-20 17:32:32.806872,2022-06-20 17:34:19.902472,0 days 00:01:47.095600,3,0.120669,0.251172,55,58,0.008318,0.377760,0.294149,0.285095,PRUNED,107.095600


In [ ]:
data = joblib.load('test_study2.pkl')

df_optuna = data.trials_dataframe()
df_optuna.dropna(inplace=True)
df_optuna.reset_index(inplace=True)

df_optuna['time'] = df_optuna.datetime_complete - df_optuna.datetime_start
df_optuna['time'] = df_optuna.time.astype('int') / (1_000_000_000)
df_optuna = df_optuna[df_optuna.time>0]

names = []

for col in df_optuna.columns.values:
    if col[1] == '':
        names.append(col[0])
    else:
        names.append(col[1])

df_optuna.columns = names

print('best val:', - round(df_optuna.value.min(),4))
a = sns.lineplot(x=df.index, y=-df_optuna.value.cummin())
a.set_xlabel('trial number')
sns.scatterplot(x=df.index, y=-df_optuna.value, color='red')
a.set_ylabel('f1 score')
a.legend(['best value', "trial's value"]);

AttributeError: ignored

In [ ]:
for idx in range(100):  # plot 10 examples
    best_tft.plot_prediction(x, raw_predictions, idx=idx, add_loss_to_title=True, show_future_observed=True);

In [ ]:
%cd /content/gdrive/My\ Drive/Forecaster/StockData/Earnings_5-20-22/Earnings_5-23-22_LimitVolume_v1_BESTCASE/1u43vr8q/checkpoints
!ls

/content/gdrive/My Drive/Forecaster/StockData/Earnings_5-20-22/Earnings_5-23-22_LimitVolume_v1_BESTCASE/1u43vr8q/checkpoints
 earnings_v5-20-22_pmi2mj2o.csv   lightning_logs   test_study2.pkl
'epoch=49-step=1499.ckpt'	  optuna_test
